In [51]:
import numpy as np
import pandas as pd
import torch
import tqdm

In [4]:
raw_data = pd.read_csv("/opt/ml/input/workspace/CF/Non-DL/FISM/user_movie_interaction.csv")
raw_data_np = raw_data.to_numpy()[:,1:]

In [40]:
unique_sid = pd.read_csv("/opt/ml/input/data/train/pro_sg/unique_sid.txt", header=None)
item_id = sorted(unique_sid.to_numpy().reshape(6807))
user_id = sorted(raw_data["user"])


In [73]:
#load model
f = open("/opt/ml/input/workspace/BPR/models/BPR.pt", 'rb')
model  = torch.load(f).to("cuda")

In [74]:
result = list()
for i in tqdm.tqdm(range(len(raw_data_np))) :
    interaction = raw_data_np[i]
    preds = list()
    user = [i]*6807
    user = torch.tensor(user).to("cuda")
    items = list(range(6807))
    items = torch.tensor(items).to("cuda")
    pred, _ = model(user, items, items)
    pred = np.array(pred.detach().cpu())
    result.append(pred)

result = np.array(result)

100%|██████████| 31360/31360 [00:42<00:00, 731.85it/s]


In [75]:
info = list()
cnt = 0
for user in result : 
    index = np.argpartition(user, -10)[-10:]
    candidates = list()
    for i in index :
        candidates.append((i,user[i]))
    candidates.sort(key=lambda x:x[1], reverse=True)
    for candidate in candidates :
        info.append((user_id[cnt], item_id[candidate[0]]))
    cnt += 1

In [76]:
info = pd.DataFrame(info, columns=['user','item'])
info.to_csv("./submission/BPR.csv",index=False)
print("Inference Done!")

Inference Done!
